https://medium.com/@pytorch_geometric/link-prediction-on-heterogeneous-graphs-with-pyg-6d5c29677c70

这里，我们尝试，不要初始特征，让初始特征变成embedding。

In [1]:
import os.path as osp
import torch
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import torch_scatter, torch_sparse, torch_cluster, torch_spline_conv, torch_geometric

In [4]:
# pip install torch==2.1.1 torch-scatter==2.1.2 torch-sparse==0.6.18 torch-cluster==1.6.3 torch-spline-conv==1.2.2 torch-geometric==2.4.0 --target=./

In [23]:
df_ori = pd.read_csv("sample.csv").head(10000)

node_order = sorted(list(set(df_ori.company_id.to_list() + df_ori.outcompany_id.to_list())))

mapping = {
    ci: idx for idx, ci in enumerate(node_order)
}
for col in df_ori:
    df_ori[col] = df_ori[col].map(mapping)

In [6]:
from torch_geometric.data import Data

data = Data(
    num_nodes = len(mapping),
    edge_index=torch.tensor(
        df_ori.T.to_numpy(), 
        dtype = torch.long
    )
)

In [7]:
data.is_directed()

True

In [8]:
n_nodes = data.num_nodes
hidden_channels = 64

In [9]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,    
)
train_data, val_data, test_data = transform(data)

ValueError: too many values to unpack (expected 2)

In [13]:
from torch_geometric.loader import LinkNeighborLoader
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[
        -1,# 10, 
        -1, 5
    ],
    neg_sampling_ratio=2.0,
    batch_size=128,
    shuffle=True,
)

NameError: name 'train_data' is not defined

In [10]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, 128)
        self.conv2 = SAGEConv(128, out_channels)
    def forward(self, x, edge_index) :
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x_from, x_to,):
        return (x_from * x_to).sum(dim=-1)

class Model(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.emb = torch.nn.Embedding(n_nodes, in_channels)
        self.gnn = GNN(in_channels, out_channels)
        self.classifier = Classifier()
    def forward(self, data):
        x_out = self.gnn(
            self.emb(data.n_id), 
            data.edge_label_index
        )
        pred = self.classifier(
            x_out[data.edge_label_index[0]], ## 边的起始点。
            x_out[data.edge_label_index[-1]] ## 边的终结点。
        )
        return pred
        
model = Model(in_channels=hidden_channels, out_channels=64)

weights = model.emb.weight.detach().numpy()
pd.DataFrame(weights, columns = [f"col_{i}" for i in range(weights.shape[1])])

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_54,col_55,col_56,col_57,col_58,col_59,col_60,col_61,col_62,col_63
0,-0.681362,0.371806,0.968334,0.332413,0.585203,-1.459547,-0.991585,-0.439894,0.745916,2.833888,...,0.706026,-0.661581,0.431125,-0.456417,-0.260911,0.503060,1.667154,0.003482,-0.655930,-0.451021
1,-0.226722,1.669456,1.395824,0.034214,-1.745908,-0.079567,-0.976496,-0.155931,-1.940745,-0.303455,...,-0.573639,0.259826,-0.595469,1.599533,0.615910,1.045948,-0.101730,1.089423,-0.217456,-0.033893
2,-0.622149,0.634217,-0.557538,0.604435,-1.052948,-0.007304,-0.253565,-1.540544,-0.501176,1.133011,...,0.951169,-0.845996,0.412613,-0.954249,0.445730,-2.001489,0.826143,1.192370,-1.307183,-0.631620
3,-0.434680,0.613189,1.300642,-0.036524,-1.300019,0.257152,1.401565,-1.179154,0.027658,0.774008,...,-0.328119,0.792314,-0.761744,0.909034,0.357677,-0.632091,0.450528,-0.709972,0.990130,-0.800832
4,-1.242741,1.545177,-0.266185,-1.144322,-0.710834,0.069757,0.085813,-1.160975,-0.948213,0.260779,...,0.610312,0.864459,-1.081501,-0.515476,1.583763,-0.214728,0.187637,0.549037,1.162275,-0.025045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,0.909331,-0.833525,0.580652,0.330278,-1.806274,0.771397,0.137043,-1.149239,-1.593335,-0.235503,...,2.089786,0.406375,-0.063296,0.064135,0.270313,-2.292699,0.148796,-0.142106,-1.114071,-1.149549
10268,-0.219491,0.394075,0.713584,0.247636,0.718843,1.308002,-0.670440,0.490052,0.141433,1.042232,...,-1.720954,-0.608537,-1.331305,0.984431,-0.944664,-1.510188,0.682498,0.619563,-1.067055,-0.885045
10269,0.223237,-0.444384,-0.534276,-0.106199,0.362217,0.241627,1.497042,0.250577,-0.234953,0.204318,...,0.080709,0.512370,-1.578883,-0.124650,0.749013,0.758084,-0.003894,0.847534,-0.759821,-1.365277
10270,0.194498,-0.988646,0.231222,-0.659330,-0.083326,-0.273684,0.705181,-0.238869,1.058705,0.099927,...,0.908556,0.655314,-0.581826,0.773552,-0.230240,2.172759,-0.087343,1.244095,-1.035697,0.845587


In [11]:
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu111.html
# import torch_geometric
# torch_geometric.typing.WITH_TORCH_SPARSE

In [12]:
import tqdm
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):        
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data.edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    
#     break
    
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")
    

Device: 'cpu'


NameError: name 'train_loader' is not defined

In [14]:
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric

In [15]:
test_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[10, 5],
    neg_sampling_ratio=2.0,
    batch_size=128,
    shuffle=True,
)

In [16]:
from sklearn.metrics import roc_auc_score
preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(test_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data.edge_label)
pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

100%|██████████████████████████████████████████| 63/63 [00:00<00:00, 401.29it/s]


Validation AUC: 0.7791


In [17]:
model.emb

Embedding(15810, 64)

In [18]:
weights = model.emb.weight.detach().numpy()
df_rst = pd.DataFrame(weights, columns = [f"col_{i}" for i in range(weights.shape[1])])
df_rst["company_id"] = node_order
df_rst.to_csv("embedding.csv", index=False)